<a href="https://colab.research.google.com/github/heispv/bioinformatics/blob/master/lab-of-bioinformatics/like_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The steps that we should take to solve the problem.

1. First we ant to select all the proteins in swiss proit with the BPTI/Kunitz domain.
    * kun + human / kun + non-human / review + human
2. Separate human from non human proteins. Use the non human proteins
as a positive in the testing set.
3. Generate a random set of negative of the same size of the positive set.
4. Remove both positives and negatives from SwissProt and perform the
prediction based on the results of the blast search

PF00014 -> kunitz in pfam

in the advanced search of the uniprot go to cross-reference -> family and domain databases -> pfam

In [1]:
!wget -O human_bpti.fasta.gz 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29%29+AND+%28reviewed%3Atrue%29+AND+%28model_organism%3A9606%29'


--2024-04-05 08:24:46--  https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29%29+AND+%28reviewed%3Atrue%29+AND+%28model_organism%3A9606%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘human_bpti.fasta.gz’

human_bpti.fasta.gz     [ <=>                ]   8.38K  --.-KB/s    in 0s      

2024-04-05 08:24:47 (80.7 MB/s) - ‘human_bpti.fasta.gz’ saved [8581]



In [2]:
!zcat -f /content/human_bpti.fasta.gz > human_bpti.fasta

In [3]:
!grep ">" /content/human_bpti.fasta | wc

     18     203    1793


In [4]:
!wget -O non_human_bpti.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29+NOT+%28organism_id%3A9606%29%29+AND+%28reviewed%3Atrue%29"


--2024-04-05 08:24:47--  https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29+NOT+%28organism_id%3A9606%29%29+AND+%28reviewed%3Atrue%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘non_human_bpti.fasta.gz’

non_human_bpti.fast     [ <=>                ]  34.71K   179KB/s    in 0.2s    

2024-04-05 08:24:48 (179 KB/s) - ‘non_human_bpti.fasta.gz’ saved [35542]



In [5]:
!zcat -f /content/non_human_bpti.fasta.gz > non_human_bpti.fasta

In [6]:
!grep ">" /content/non_human_bpti.fasta | wc

    373    3715   38651


In [7]:
!wget -O reviewd_human.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28reviewed%3Atrue%29+AND+%28organism_id%3A9606%29%29"


--2024-04-05 08:24:48--  https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28reviewed%3Atrue%29+AND+%28organism_id%3A9606%29%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘reviewd_human.fasta.gz’

reviewd_human.fasta     [   <=>              ]   7.16M   562KB/s    in 18s     

2024-04-05 08:25:07 (408 KB/s) - ‘reviewd_human.fasta.gz’ saved [7505825]



In [8]:
!zcat -f /content/reviewd_human.fasta.gz > reviewd_human.fasta

In [9]:
!grep ">" /content/reviewd_human.fasta | wc

  20434  224167 2053803


We want to use the BLAST and the reference would be the human one.

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt upgrade

In [ ]:
!sudo apt-get install ncbi-blast+

In [45]:
!makeblastdb -in "/content/human_bpti.fasta" -dbtype prot



Building a new DB, current time: 04/05/2024 08:51:29
New DB name:   /content/human_bpti.fasta
New DB title:  /content/human_bpti.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /content/human_bpti.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 18 sequences in 0.00214815 seconds.




In [46]:
!blastp -query "/content/non_human_bpti.fasta" -db "/content/human_bpti.fasta" -out non-human-bpti.blast -outfmt 7

In [47]:
!grep -v "^#" non-human-bpti.blast | head -n 50

sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	43.636	55	31	0	1	55	121	175	2.98e-16	61.2
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	40.741	54	32	0	3	56	52	105	1.23e-14	57.0
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	40.741	54	32	0	2	55	214	267	4.03e-11	47.0
sp|A0A1Z0YU59|MAMB1_DENAN	sp|Q02388|CO7A1_HUMAN	46.296	54	26	1	5	55	2876	2929	8.17e-15	57.8
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P12111|CO6A3_HUMAN	41.176	51	30	0	5	55	3112	3162	1.77e-14	56.6
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	45.283	53	29	0	3	55	34	86	3.12e-14	55.5
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	40.000	50	30	0	2	51	155	204	9.09e-10	43.1
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	29.310	58	38	2	2	56	93	150	0.005	24.3
sp|A0A1Z0YU59|MAMB1_DENAN	sp|O95925|EPPI_HUMAN	40.000	55	33	0	1	55	73	127	3.41e-14	53.9
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P49223|SPIT3_HUMAN	38.889	54	33	0	2	55	33	86	4.53e-14	52.4
sp|A0A1Z0YU59|MAMB1_DENAN	sp|Q8TEU8|WFKN2_HUMAN	43.137	51	29	0	5	55	386	436	5.59e-14	55.1
sp|A0A1Z0YU59|MAM

In [48]:
!grep -v "^#" non-human-bpti.blast | cut -f 1 | sort | uniq -c | head -n 20

     48 sp|A0A1D0BND9|TD1A_HADIN
     27 sp|A0A1Z0YU59|MAMB1_DENAN
     30 sp|A0A3G2FQK2|3DPD_HETMG
     31 sp|A0A6B7FA07|VKT2J_HETCR
     32 sp|A0A6B7FBD3|VKT2I_HETCR
     31 sp|A0A6B7FEJ3|VKT2K_HETCR
     72 sp|A0A6P8HC43|ATPI1_ACTTE
     34 sp|A5X2X1|VKT_SISCA
     31 sp|A6MFL1|VKT1_DEMVE
     32 sp|A6MFL2|VKT2_DEMVE
     30 sp|A6MFL3|VKT3_DEMVE
     28 sp|A6MFL4|VKT4_DEMVE
     30 sp|A6MGX9|VKT5_DEMVE
     31 sp|A6MGY1|VKT7_DEMVE
     31 sp|A7X3V4|VKT1_TELDH
     31 sp|A7X3V7|VKT1_PHIOL
     29 sp|A8Y7N4|VKTC1_DABSI
     29 sp|A8Y7N5|VKTC2_DABSI
     32 sp|A8Y7N6|VKTC3_DABSI
     32 sp|A8Y7N7|VKTC4_DABSI


because BLAST is based on the local alignment, if the inputs are long maybe there are 2 matches between the first sequence and first seq in the blastdb

In [49]:
!grep -v "^#" non-human-bpti.blast | cut -f 1 | sort -u | cut -d "|" -f 2 > non-human-bpti.ids

In [50]:
!cat non-human-bpti.ids | head -n 10

A0A1D0BND9
A0A1Z0YU59
A0A3G2FQK2
A0A6B7FA07
A0A6B7FBD3
A0A6B7FEJ3
A0A6P8HC43
A5X2X1
A6MFL1
A6MFL2


In [51]:
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | head -n 1; done | wc

    373    4476   32107


In [52]:
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | sort -nrk 11; done | head -n 10

sp|A0A1D0BND9|TD1A_HADIN	sp|P02760|AMBP_HUMAN	36.765	68	41	1	16	81	277	344	8.31e-06	34.7
sp|A0A1D0BND9|TD1A_HADIN	sp|P02760|AMBP_HUMAN	39.560	91	51	1	32	122	237	323	8.24e-14	57.8
sp|A0A1D0BND9|TD1A_HADIN	sp|O43278|SPIT1_HUMAN	30.645	62	43	0	17	78	241	302	7.91e-06	34.7
sp|A0A1D0BND9|TD1A_HADIN	sp|P49223|SPIT3_HUMAN	48.077	52	26	1	71	122	22	72	7.71e-12	49.3
sp|A0A1D0BND9|TD1A_HADIN	sp|P49223|SPIT3_HUMAN	38.182	55	34	0	22	76	32	86	6.68e-09	41.6
sp|A0A1D0BND9|TD1A_HADIN	sp|Q6UDR6|SPIT4_HUMAN	46.512	43	23	0	81	123	36	78	6.60e-10	44.3
sp|A0A1D0BND9|TD1A_HADIN	sp|P12111|CO6A3_HUMAN	39.583	48	29	0	75	122	3101	3148	6.41e-11	49.7
sp|A0A1D0BND9|TD1A_HADIN	sp|Q06481|APLP2_HUMAN	47.619	42	22	0	81	122	305	346	6.16e-12	52.4
sp|A0A1D0BND9|TD1A_HADIN	sp|P05067|A4_HUMAN	50.000	34	17	0	89	122	294	327	6.11e-10	46.6
sp|A0A1D0BND9|TD1A_HADIN	sp|P12111|CO6A3_HUMAN	40.000	10	6	0	84	93	2589	2598	5.9	17.7


In [53]:
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | sort -gk 11; done | head -n 10

sp|A0A1D0BND9|TD1A_HADIN	sp|P10646|TFPI1_HUMAN	37.255	102	53	1	33	123	61	162	1.18e-18	70.9
sp|A0A1D0BND9|TD1A_HADIN	sp|O43291|SPIT2_HUMAN	27.737	137	64	2	21	122	33	169	1.33e-15	62.0
sp|A0A1D0BND9|TD1A_HADIN	sp|P48307|TFPI2_HUMAN	37.143	105	65	1	19	122	29	133	2.33e-15	61.2
sp|A0A1D0BND9|TD1A_HADIN	sp|Q8TEU8|WFKN2_HUMAN	33.673	98	61	2	26	122	328	422	1.12e-14	60.1
sp|A0A1D0BND9|TD1A_HADIN	sp|P02760|AMBP_HUMAN	39.560	91	51	1	32	122	237	323	8.24e-14	57.8
sp|A0A1D0BND9|TD1A_HADIN	sp|P10646|TFPI1_HUMAN	29.008	131	61	2	24	122	123	253	1.57e-13	56.6
sp|A0A1D0BND9|TD1A_HADIN	sp|Q06481|APLP2_HUMAN	39.655	58	35	0	19	76	303	360	3.38e-12	53.1
sp|A0A1D0BND9|TD1A_HADIN	sp|P12111|CO6A3_HUMAN	39.623	53	32	0	24	76	3110	3162	4.44e-12	52.8
sp|A0A1D0BND9|TD1A_HADIN	sp|Q06481|APLP2_HUMAN	47.619	42	22	0	81	122	305	346	6.16e-12	52.4
sp|A0A1D0BND9|TD1A_HADIN	sp|P49223|SPIT3_HUMAN	48.077	52	26	1	71	122	22	72	7.71e-12	49.3


In [54]:
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | sort -rgk 11 | head -n 1; done | head -n 10

sp|A0A1D0BND9|TD1A_HADIN	sp|P12111|CO6A3_HUMAN	40.000	10	6	0	84	93	2589	2598	5.9	17.7
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	29.310	58	38	2	2	56	93	150	0.005	24.3
sp|A0A3G2FQK2|3DPD_HETMG	sp|P12111|CO6A3_HUMAN	50.000	8	4	0	33	40	1485	1492	9.4	15.8
sp|A0A6B7FA07|VKT2J_HETCR	sp|P12111|CO6A3_HUMAN	50.000	8	4	0	33	40	1485	1492	9.1	15.8
sp|A0A6B7FBD3|VKT2I_HETCR	sp|Q2UY09|COSA1_HUMAN	25.000	40	19	1	5	33	6	45	8.9	16.2
sp|A0A6B7FEJ3|VKT2K_HETCR	sp|P12111|CO6A3_HUMAN	34.615	26	14	1	18	40	1467	1492	8.0	16.2
sp|A0A6P8HC43|ATPI1_ACTTE	sp|P12111|CO6A3_HUMAN	54.545	11	5	0	44	54	497	507	7.6	18.9
sp|A5X2X1|VKT_SISCA	sp|P12111|CO6A3_HUMAN	36.842	19	12	0	92	110	2748	2766	8.5	17.3
sp|A6MFL1|VKT1_DEMVE	sp|O95428|PPN_HUMAN	22.857	35	19	1	16	42	32	66	9.9	16.2
sp|A6MFL2|VKT2_DEMVE	sp|P12111|CO6A3_HUMAN	40.000	20	5	1	40	52	775	794	9.4	16.2


In [55]:
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | sort -gk 11 | head -n 1; done > non-human-bpti.match

In [56]:
!grep ">" reviewd_human.fasta | wc

  20434  224167 2053803


In [57]:
!du -h reviewd_human.fasta

14M	reviewd_human.fasta


You can download the script from the github :)

In [58]:
!wget https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/like-project-script1.py

--2024-04-05 08:51:47--  https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/like-project-script1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 639 [text/plain]
Saving to: ‘like-project-script1.py.1’

like-project-script 100%[===================>]     639  --.-KB/s    in 0s      

2024-04-05 08:51:47 (43.2 MB/s) - ‘like-project-script1.py.1’ saved [639/639]



In [59]:
!python3 like-project-script1.py non-human-bpti.ids non_human_bpti.fasta > human_nobpti_r1.fasta

In [60]:
!grep ">" human_nobpti_r1.fasta | wc

    373     373    3012


remember these two codes:
```bash
!for i in `cat non-human-bpti.ids`; do grep "|"$i"|" non-human-bpti.blast | grep -v "^#" | sort -gk 11 | head -n 1; done > non-human-bpti.match
```
and
```bash
!blastp -query "/content/non_human_bpti.fasta" -db "/content/human_bpti.fasta" -out non-human-bpti.blast -outfmt 7
```

In [61]:
!cat non-human-bpti.blast | head -n 20

# BLASTP 2.12.0+
# Query: sp|A0A1Z0YU59|MAMB1_DENAN Mambaquaretin-1 OS=Dendroaspis angusticeps OX=8618 PE=1 SV=2
# Database: /content/human_bpti.fasta
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 27 hits found
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	43.636	55	31	0	1	55	121	175	2.98e-16	61.2
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	40.741	54	32	0	3	56	52	105	1.23e-14	57.0
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P10646|TFPI1_HUMAN	40.741	54	32	0	2	55	214	267	4.03e-11	47.0
sp|A0A1Z0YU59|MAMB1_DENAN	sp|Q02388|CO7A1_HUMAN	46.296	54	26	1	5	55	2876	2929	8.17e-15	57.8
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P12111|CO6A3_HUMAN	41.176	51	30	0	5	55	3112	3162	1.77e-14	56.6
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	45.283	53	29	0	3	55	34	86	3.12e-14	55.5
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	40.000	50	30	0	2	51	155	204	9.09e-10	43.1
sp|A0A1Z0YU59|MAMB1_DENAN	sp|P48307|TFPI2_HUMAN	29.31

In [62]:
!wc non-human-bpti.match

  373  4476 32109 non-human-bpti.match


In [63]:
!sort -grk 11 non-human-bpti.match | head -n 20

sp|D3GGZ8|BLI5_HAECO	sp|Q2UY09|COSA1_HUMAN	23.944	71	41	4	125	191	1062	1123	0.045	25.0
sp|O62247|BLI5_CAEEL	sp|Q96NZ8|WFKN1_HUMAN	29.167	72	48	2	115	184	339	409	2.44e-04	32.0
sp|Q8WPG5|KUNI_ORNKA	sp|P12111|CO6A3_HUMAN	37.143	35	21	1	97	130	3129	3163	4.05e-05	33.1
sp|Q11101|YL15_CAEEL	sp|P05067|A4_HUMAN	37.500	40	25	0	38	77	305	344	1.85e-07	41.2
sp|P86963|KCP2_PINMA	sp|P12111|CO6A3_HUMAN	36.207	58	30	2	98	153	3112	3164	1.33e-07	43.9
sp|D9IFL3|KUN1_RHIHE	sp|O43278|SPIT1_HUMAN	34.722	72	37	2	67	138	380	441	8.35e-08	41.6
sp|P0CAR0|VKTG1_MICPY	sp|P12111|CO6A3_HUMAN	44.828	29	16	0	6	34	3112	3140	6.47e-08	37.0
sp|Q09JW6|KUNI_ARGMO	sp|Q6UDR6|SPIT4_HUMAN	36.667	60	35	3	80	139	35	91	2.04e-08	40.8
sp|P0CH75|VKT25_CYRHA	sp|P12111|CO6A3_HUMAN	44.118	34	17	1	2	33	3110	3143	1.01e-09	42.0
sp|Q2ES49|VKT2_DABRR	sp|P10646|TFPI1_HUMAN	38.596	57	35	0	25	81	119	175	6.96e-10	44.7
sp|G9I929|VKTA_MICTN	sp|P10646|TFPI1_HUMAN	35.484	62	35	2	24	83	118	176	6.31e-10	45.1
sp|C0LNR2|VKT_VESBI	sp|O95428|PPN_HUMAN	38.7

note that the evalue sometimes are higher than expected, because the dataset is not large enough.

In [79]:
!blastp -query human_nobpti_r1.fasta -db human_bpti.fasta -out human_nobpti_r1.match -outfmt 7

In [80]:
!cat human_nobpti_r1.match | head -n 20

# BLASTP 2.12.0+
# Query: A0A1D0BND9
# Database: human_bpti.fasta
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 48 hits found
A0A1D0BND9	sp|P10646|TFPI1_HUMAN	37.255	102	53	1	33	123	61	162	1.18e-18	70.9
A0A1D0BND9	sp|P10646|TFPI1_HUMAN	29.008	131	61	2	24	122	123	253	1.57e-13	56.6
A0A1D0BND9	sp|P10646|TFPI1_HUMAN	43.137	51	29	0	26	76	217	267	2.71e-08	42.0
A0A1D0BND9	sp|O43291|SPIT2_HUMAN	27.737	137	64	2	21	122	33	169	1.33e-15	62.0
A0A1D0BND9	sp|O43291|SPIT2_HUMAN	32.558	43	29	0	24	66	131	173	5.63e-05	32.3
A0A1D0BND9	sp|P48307|TFPI2_HUMAN	37.143	105	65	1	19	122	29	133	2.33e-15	61.2
A0A1D0BND9	sp|P48307|TFPI2_HUMAN	31.250	112	71	3	15	122	85	194	2.81e-11	50.1
A0A1D0BND9	sp|P48307|TFPI2_HUMAN	42.308	52	30	0	21	72	153	204	1.18e-09	45.4
A0A1D0BND9	sp|Q8TEU8|WFKN2_HUMAN	33.673	98	61	2	26	122	328	422	1.12e-14	60.1
A0A1D0BND9	sp|Q8TEU8|WFKN2_HUMAN	38.462	52	32	0	26	77	386	437	1.21e-08	43.1
A0

* I am lost here now :|

In [81]:
!for i in `cat like-project-script1.py`; do grep "|"$i"|" human_nobpti_r1.match | grep -v "^#" | sort -gk 11 | head -n 1; done > human_nobpti_r1.match

In [82]:
!cat human_nobpti_r1.match | head -n 20

We have the `nonhuman-bpti.match` and `human-nonbpti.match`. we should check what is the best threshold for the `e-value` for the classification of the problem.

You can search in the negative examples, and olnly select the ones which have the lowest e-value, so in this way we are putting the predictor in a criticial way.